# Fire Detection Using CNN with Weight Optimization

## Importing Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from datetime import datetime

## Loading Dataset

In [ ]:
BASE_DIR = "fire_dataset/"   # rename if different
FIRE_DIR = os.path.join(BASE_DIR, "fire_images")
NONFIRE_DIR = os.path.join(BASE_DIR, "non_fire_images")

print("Fire images:", len(os.listdir(FIRE_DIR)))
print("Non-fire images:", len(os.listdir(NONFIRE_DIR)))

## Data Preprocessing and Image Data Generator

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,   # ensures equal split
)
train_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

In [ ]:
print("Class mapping:", train_gen.class_indices)
idx_to_label = {v: k for k, v in train_gen.class_indices.items()}
idx_to_label

## Model compilation and Class Weight Balancing

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

y_classes = train_gen.classes

weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_classes),
    y=y_classes
)

class_weights = {i: w for i, w in enumerate(weights)}
print("Class Weights:", class_weights)

In [ ]:
class_weights

## CNN Model Architecture

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),

    layers.Dense(1, activation='sigmoid')   # Binary output
])

model.compile(
    optimizer=Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


## Tensorboard Integration

In [ ]:
log_dir = "logs/fire_detect_" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,          # 🔥 enable weight histograms
    write_graph=True,          # 🔥 enable model graph
    write_images=True,         # 🔥 write conv filter images
    update_freq='batch',       # 🔥 more frequent logging
    profile_batch=5            # 🔥 profile performance on batch 5
)

print("Tensorboard logging to:", log_dir)

## Model Training

In [ ]:
history = model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[tensorboard_callback]
)

## Visualizations

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.legend(["Train", "Validation"])

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.legend(["Train", "Validation"])

plt.show()

In [ ]:
val_gen_cm = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode="binary",
    subset="validation",
    shuffle=False
)
y_true = val_gen_cm.classes   # array of 0s and 1s
class_labels = list(val_gen_cm.class_indices.keys())
print("Class labels:", class_labels)

y_pred_prob = model.predict(val_gen_cm)
y_pred = (y_pred_prob > 0.5).astype("int32")  # convert to 0/1

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix — Fire Detection CNN")
plt.show()

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_labels))

In [ ]:
tn, fp, fn, tp = cm.ravel()

fire_recall = tp / (tp + fn)
nonfire_recall = tn / (tn + fp)

print("🔥 Fire Recall (true positive rate): ", fire_recall)
print("🌿 Non-Fire Recall (true negative rate): ", nonfire_recall)

## Predictions and Results

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_image(path):
    img = image.load_img(path, target_size=IMG_SIZE)
    img_arr = image.img_to_array(img) / 255.0
    img_arr = np.expand_dims(img_arr, axis=0)

    prob = model.predict(img_arr)[0][0]
    pred_class = int(prob > 0.5)
    folder_name = idx_to_label[pred_class]

    plt.imshow(img)
    plt.axis("off")

    if "fire" in folder_name:
        plt.title(f"🔥 FIRE DETECTED (prob={prob:.2f})")
    else:
        plt.title(f"🌿 NON-FIRE (prob={prob:.2f})")

    plt.show()

In [ ]:
import random

def demo_grid(folder, n=6):
    files = random.sample(os.listdir(folder), n)
    plt.figure(figsize=(15,8))

    for i, f in enumerate(files):
        full_path = os.path.join(folder, f)

        img = image.load_img(full_path, target_size=IMG_SIZE)
        img_arr = image.img_to_array(img) / 255.0
        img_arr = np.expand_dims(img_arr, axis=0)
        
        prob = model.predict(img_arr)[0][0]
        pred_class = int(prob > 0.5)
        label = idx_to_label[pred_class]

        plt.subplot(2, n//2, i+1)
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"{label}\n({prob:.2f})")

    plt.tight_layout()
    plt.show()


## Model Saving

In [ ]:
model.save("fire_detection_cnn")